In [ ]:
# We will be attempting to predict future NBA Hall of Famers
# We will be training out machine learning model using certain advance statistics categories

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs

In [3]:
# Read in CSV with all player data
raw_data = pd.read_csv('Seasons_Stats.csv')
raw_data.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [4]:
# Look at our data to see column names
raw_data.columns.values

array(['Unnamed: 0', 'Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS',
       'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%',
       'STL%', 'BLK%', 'TOV%', 'USG%', 'blanl', 'OWS', 'DWS', 'WS',
       'WS/48', 'blank2', 'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS'], dtype=object)

In [25]:
# Drop columns with no data
raw_data = raw_data.drop(['Unnamed: 0','blanl','blank2'],axis=1)

In [28]:
raw_data.shape

(24691, 50)

In [23]:
# To accuratley predict future Hall of Famers we want to only include players that have
# certain advance statistics, we want to omit players that do not have any values in
# 'WS', 'BPM', 'VORP', 'TRB', 'AST', 'STL', 'BLK', 'PTS'
drop_players = []
for i in range(len(raw_data)):
    if np.isnan(raw_data['WS'][i]) == True\
    or np.isnan(raw_data['BPM'][i]) == True\
    or np.isnan(raw_data['VORP'][i]) == True\
    or np.isnan(raw_data['TRB'][i]) == True\
    or np.isnan(raw_data['AST'][i]) == True\
    or np.isnan(raw_data['STL'][i]) == True\
    or np.isnan(raw_data['BLK'][i]) == True\
    or np.isnan(raw_data['PTS'][i]) == True:
        drop_players.append(raw_data['Player'][i])
drop_players = np.unique(drop_players)

In [24]:
# Count the number of players that will be dropped
drop_players.shape

(977,)

In [27]:
raw_data.shape

(24691, 50)

In [31]:
# Create a dataset that will only include 
# the statistics I want to use to train the model
clean_data = raw_data[['Player','G','GS','MP','FG','WS','BPM','VORP','FGA','3P','3PA','2P','2PA','FT','FTA','ORB','DRB','TRB','AST','STL','BLK','TOV','PTS']]
clean_data.head()

,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,3P,...,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PTS
0,Curly Armstrong,63.0,NaN,NaN,144.0,3.5,NaN,NaN,516.0,NaN,...,170.0,241.0,NaN,NaN,NaN,176.0,NaN,NaN,NaN,458.0
1,Cliff Barker,49.0,NaN,NaN,102.0,2.2,NaN,NaN,274.0,NaN,...,75.0,106.0,NaN,NaN,NaN,109.0,NaN,NaN,NaN,279.0
2,Leo Barnhorst,67.0,NaN,NaN,174.0,3.6,NaN,NaN,499.0,NaN,...,90.0,129.0,NaN,NaN,NaN,140.0,NaN,NaN,NaN,438.0
3,Ed Bartels,15.0,NaN,NaN,22.0,-0.6,NaN,NaN,86.0,NaN,...,19.0,34.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,63.0
4,Ed Bartels,13.0,NaN,NaN,21.0,-0.6,NaN,NaN,82.0,NaN,...,17.0,31.0,NaN,NaN,NaN,20.0,NaN,NaN,NaN,59.0


In [32]:
clean_data.shape

(24691, 23)

In [33]:
# Use 'Group By' to add all the season statistics together
clean_data = clean_data.groupby('Player').sum().reset_index()

In [34]:
clean_data.shape

(3921, 23)

In [35]:
clean_data

,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,3P,...,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PTS
0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,125.0,...,3247.0,4447.0,3576.0,6553.0,10129.0,1469.0,1103.0,562.0,1508.0,12928.0
1,A.J. Bramlett,8.0,0.0,61.0,4.0,-0.2,-16.8,-0.2,21.0,0.0,...,0.0,0.0,12.0,10.0,22.0,0.0,1.0,0.0,3.0,8.0
2,A.J. English,151.0,18.0,3108.0,617.0,1.1,-10.2,-2.4,1418.0,9.0,...,259.0,333.0,140.0,175.0,315.0,320.0,57.0,24.0,203.0,1502.0
3,A.J. Guyton,80.0,14.0,1246.0,166.0,0.3,-33.0,-1.0,440.0,73.0,...,37.0,45.0,22.0,58.0,80.0,147.0,20.0,12.0,62.0,442.0
4,A.J. Hammons,22.0,0.0,163.0,17.0,0.0,-5.6,-0.1,42.0,5.0,...,9.0,20.0,8.0,28.0,36.0,4.0,1.0,13.0,10.0,48.0
5,A.J. Price,287.0,25.0,4253.0,600.0,5.0,-37.3,-0.5,1581.0,233.0,...,223.0,303.0,74.0,335.0,409.0,613.0,132.0,9.0,243.0,1656.0
6,A.J. Wynder,6.0,0.0,39.0,3.0,0.0,-9.4,-0.1,12.0,0.0,...,6.0,8.0,1.0,2.0,3.0,8.0,1.0,0.0,4.0,12.0
7,A.W. Holt,6.0,0.0,14.0,1.0,-0.1,0.0,0.0,8.0,0.0,...,2.0,3.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0
8,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,6893.0,1025.0,...,1156.0,1371.0,323.0,1037.0,1360.0,2485.0,469.0,113.0,1272.0,7839.0
9,Aaron Gordon,205.0,117.0,4958.0,760.0,10.1,-1.7,2.4,1652.0,132.0,...,329.0,471.0,316.0,765.0,1081.0,311.0,144.0,117.0,193.0,1981.0


In [87]:
# Drop the players from our 'drop_player' list
# Create new dataset
for i in drop_players:
    new_data = clean_data[clean_data.Player.str.contains(i) == False]

new_data = new_data.reset_index()

In [88]:
new_data.shape

(3913, 24)

In [89]:
HOF = []
for i in range(len(new_data)):
    if '*' in new_data.iloc[i]['Player']:
        HOF.append(1)
    else:
        HOF.append(0)


In [90]:
new_data['HOF'] = HOF
new_data.head()

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PTS,HOF
0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,...,4447.0,3576.0,6553.0,10129.0,1469.0,1103.0,562.0,1508.0,12928.0,0
1,1,A.J. Bramlett,8.0,0.0,61.0,4.0,-0.2,-16.8,-0.2,21.0,...,0.0,12.0,10.0,22.0,0.0,1.0,0.0,3.0,8.0,0
2,2,A.J. English,151.0,18.0,3108.0,617.0,1.1,-10.2,-2.4,1418.0,...,333.0,140.0,175.0,315.0,320.0,57.0,24.0,203.0,1502.0,0
3,3,A.J. Guyton,80.0,14.0,1246.0,166.0,0.3,-33.0,-1.0,440.0,...,45.0,22.0,58.0,80.0,147.0,20.0,12.0,62.0,442.0,0
4,4,A.J. Hammons,22.0,0.0,163.0,17.0,0.0,-5.6,-0.1,42.0,...,20.0,8.0,28.0,36.0,4.0,1.0,13.0,10.0,48.0,0


In [91]:
# Drop columns with name 'index'
new_data = new_data.drop(['index'],axis=1)

In [92]:
new_data[new_data.HOF==1]

,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,3P,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PTS,HOF
29,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,17203.0,7.0,...,9599.0,2589.0,3803.0,6392.0,3254.0,1105.0,187.0,2894.0,26274.0,1
36,Al Cervi*,202.0,0.0,1151.0,405.0,22.0,0.0,0.0,1128.0,0.0,...,931.0,0.0,0.0,261.0,648.0,0.0,0.0,0.0,1591.0,1
72,Alex English*,1271.0,753.0,40464.0,11212.0,105.9,20.0,37.4,22149.0,20.0,...,5407.0,3047.0,4096.0,7143.0,4575.0,1140.0,895.0,3035.0,26931.0,1
75,Alex Hannum*,516.0,0.0,7713.0,1182.0,11.4,0.0,0.0,3341.0,0.0,...,1135.0,0.0,0.0,2013.0,857.0,0.0,0.0,0.0,3078.0,1
86,Alfred McGuire*,191.0,0.0,2966.0,251.0,5.4,0.0,0.0,663.0,0.0,...,463.0,0.0,0.0,418.0,363.0,0.0,0.0,0.0,757.0,1
92,Allen Iverson*,1064.0,1042.0,43303.0,9532.0,108.7,20.7,46.0,22369.0,1158.0,...,9256.0,814.0,3025.0,3839.0,6488.0,2213.0,185.0,3740.0,27457.0,1
98,Alonzo Mourning*,875.0,703.0,26677.0,5216.0,90.9,4.1,23.9,9913.0,22.0,...,6003.0,2237.0,5098.0,7335.0,964.0,422.0,2430.0,2266.0,14593.0,1
149,Andy Phillip*,609.0,0.0,15378.0,1895.0,42.3,0.0,0.0,5075.0,0.0,...,2177.0,0.0,0.0,2395.0,3366.0,0.0,0.0,0.0,5320.0,1
204,Arnie Risen*,577.0,0.0,12690.0,2265.0,45.3,0.0,0.0,6034.0,0.0,...,2989.0,0.0,0.0,5011.0,958.0,0.0,0.0,0.0,6638.0,1
215,Artis Gilmore*,980.0,503.0,30578.0,5831.0,108.4,23.2,39.1,9751.0,1.0,...,5896.0,2708.0,6664.0,9372.0,1798.0,485.0,1777.0,2414.0,15844.0,1


In [93]:
# Looking at the NBA player who made the Hall of Fame
# There are players who made the Hall of Fame as coaches
# Other players made them as overseas players
# To make sure our dataset will result properly
# We will drop all players with less than 450 games played
# And with less than 6000 points scored
player_game = []
for i in range(len(new_data)):
    if new_data['G'][i] < 450 or new_data['PTS'][i] < 6000:
        player_game.append(new_data['Player'][i])

for i in player_game:
    new_data = new_data[new_data.Player.str.match(i) == False]

In [54]:
player_list = pd.read_csv('allPlayers.csv')
player_list.head()

,Player,URL,From,To,Pos,Ht,Wt,Birth Date,Colleges
0,Alaa Abdelnaby,abdelal01,1991,1995,F-C,10-Jun,240.0,June 24 1968,Duke University
1,Zaid Abdul-Aziz,abdulza01,1969,1978,C-F,9-Jun,235.0,April 7 1946,Iowa State University
2,Kareem Abdul-Jabbar*,abdulka01,1970,1989,C,2-Jul,225.0,April 16 1947,University of California Los Angeles
3,Mahmoud Abdul-Rauf,abdulma02,1991,2001,G,1-Jun,162.0,March 9 1969,Louisiana State University
4,Tariq Abdul-Wahad,abdulta01,1998,2003,F,6-Jun,223.0,November 3 1974,University of Michigan San Jose State University


In [60]:
# Add Award Categories
new_data['all_star_appear'] = np.nan
new_data['nba_champ'] = np.nan
new_data['all_defensive']= np.nan
new_data['all_star_mvp'] = np.nan
new_data['league_mvp'] = np.nan
new_data['scoring_champ'] = np.nan
new_data['all_nba'] = np.nan
new_data['finals_mvp'] = np.nan
new_data['def_poy'] = np.nan
new_data['stl_champ'] = np.nan
new_data['ast_champ'] = np.nan
new_data['trb_champ'] = np.nan
new_data['blk_champ'] = np.nan

KeyError: 0

In [68]:
import re
import requests
for counter, i in enumerate(new_data['Player']):
    if counter == 3913:
        url = 'https://www.basketball-reference.com/players/h/hilarne01.html'
    else:
        stem = player_list[player_list['Player'].str.contains(i)].reset_index().at[0,'URL']
        url = 'https://www.basketball-reference.com/players/'+stem[0]+'/'+stem+'.html'
    page = requests.get(url)
    soup = bs(page.content,'html.parser')
    if soup.find("ul",id='bling') is None:
        new_data.iloc[counter,25:] = 0
        continue
    else:
        body = soup.find("ul", id='bling').findAll('a')
    for award in body:
        result = re.sub('[^0-9]','', award.getText())
        if('All Star' in award.getText()): 
            new_data.at[new_data.index[counter], 'all_star_appear'] = result
            continue
        elif('NBA Champ' in award.getText()):
            new_data.at[new_data.index[counter], 'nba_champ'] = result
            continue
        elif('All-Defensive' in award.getText()):
            new_data.at[new_data.index[counter], 'all_defensive'] = result
            continue
        elif('AS MVP' in award.getText()):
            new_data.at[new_data.index[counter], 'all_star_mvp'] = result
            continue
        elif('Scoring Champ' in award.getText()):
            new_data.at[new_data.index[counter], 'scoring_champ'] = result
            continue
        elif('All-NBA' in award.getText()):
            new_data.at[new_data.index[counter], 'all_nba'] = result
            continue
        elif('Finals MVP' in award.getText()):
            new_data.at[new_data.index[counter], 'finals_mvp'] = result
            continue
        elif('STL Champ' in award.getText()):
            new_data.at[new_data.index[counter], 'stl_champ'] = result
            continue
        elif('TRB Champ' in award.getText()):
            new_data.at[new_data.index[counter], 'trb_champ'] = result
            continue
        elif('BLK Champ' in award.getText()):
            new_data.at[new_data.index[counter], 'blk_champ'] = result
            continue
        elif('AST Champ' in award.getText()):
            new_data.at[new_data.index[counter], 'ast_champ'] = result
            continue
        elif('Def. POY' in award.getText()):
            new_data.at[new_data.index[counter], 'def_poy'] = result
            continue

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PTS,HOF
0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,...,4447.0,3576.0,6553.0,10129.0,1469.0,1103.0,562.0,1508.0,12928.0,0
1,1,A.J. Bramlett,8.0,0.0,61.0,4.0,-0.2,-16.8,-0.2,21.0,...,0.0,12.0,10.0,22.0,0.0,1.0,0.0,3.0,8.0,0
2,2,A.J. English,151.0,18.0,3108.0,617.0,1.1,-10.2,-2.4,1418.0,...,333.0,140.0,175.0,315.0,320.0,57.0,24.0,203.0,1502.0,0
3,3,A.J. Guyton,80.0,14.0,1246.0,166.0,0.3,-33.0,-1.0,440.0,...,45.0,22.0,58.0,80.0,147.0,20.0,12.0,62.0,442.0,0
4,4,A.J. Hammons,22.0,0.0,163.0,17.0,0.0,-5.6,-0.1,42.0,...,20.0,8.0,28.0,36.0,4.0,1.0,13.0,10.0,48.0,0
5,5,A.J. Price,287.0,25.0,4253.0,600.0,5.0,-37.3,-0.5,1581.0,...,303.0,74.0,335.0,409.0,613.0,132.0,9.0,243.0,1656.0,0
6,6,A.J. Wynder,6.0,0.0,39.0,3.0,0.0,-9.4,-0.1,12.0,...,8.0,1.0,2.0,3.0,8.0,1.0,0.0,4.0,12.0,0
7,7,A.W. Holt,6.0,0.0,14.0,1.0,-0.1,0.0,0.0,8.0,...,3.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0
8,8,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,6893.0,...,1371.0,323.0,1037.0,1360.0,2485.0,469.0,113.0,1272.0,7839.0,0
9,9,Aaron Gordon,205.0,117.0,4958.0,760.0,10.1,-1.7,2.4,1652.0,...,471.0,316.0,765.0,1081.0,311.0,144.0,117.0,193.0,1981.0,0


In [135]:
new_data[new_data.HOF==1]

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PTS,HOF,all_star_appear
3,3,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,17203.0,...,2589.0,3803.0,6392.0,3254.0,1105.0,187.0,2894.0,26274.0,1,6
10,10,Alex English*,1271.0,753.0,40464.0,11212.0,105.9,20.0,37.4,22149.0,...,3047.0,4096.0,7143.0,4575.0,1140.0,895.0,3035.0,26931.0,1,11
12,12,Allen Iverson*,1064.0,1042.0,43303.0,9532.0,108.7,20.7,46.0,22369.0,...,814.0,3025.0,3839.0,6488.0,2213.0,185.0,3740.0,27457.0,1,7
14,14,Alonzo Mourning*,875.0,703.0,26677.0,5216.0,90.9,4.1,23.9,9913.0,...,2237.0,5098.0,7335.0,964.0,422.0,2430.0,2266.0,14593.0,1,1
36,36,Arnie Risen*,577.0,0.0,12690.0,2265.0,45.3,0.0,0.0,6034.0,...,0.0,0.0,5011.0,958.0,0.0,0.0,0.0,6638.0,1,NaN
38,38,Artis Gilmore*,980.0,503.0,30578.0,5831.0,108.4,23.2,39.1,9751.0,...,2708.0,6664.0,9372.0,1798.0,485.0,1777.0,2414.0,15844.0,1,6
42,42,Bailey Howell*,950.0,0.0,30627.0,6515.0,114.7,0.0,0.0,13585.0,...,0.0,0.0,9383.0,1853.0,0.0,0.0,0.0,17770.0,1,0
48,48,Bernard King*,874.0,547.0,29417.0,7830.0,75.2,12.8,27.5,15109.0,...,1704.0,3356.0,5060.0,2863.0,866.0,230.0,2791.0,19655.0,1,0
49,49,Bill Bradley*,742.0,0.0,22799.0,3927.0,38.8,-10.1,-0.8,8763.0,...,198.0,643.0,2354.0,2533.0,209.0,65.0,0.0,9217.0,1,12
54,54,Bill Russell*,963.0,0.0,40726.0,5687.0,163.4,0.0,0.0,12930.0,...,0.0,0.0,21620.0,4100.0,0.0,0.0,0.0,14522.0,1,0


In [136]:
new_data

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PTS,HOF,all_star_appear
0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,...,3576.0,6553.0,10129.0,1469.0,1103.0,562.0,1508.0,12928.0,0,1
1,1,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,6893.0,...,323.0,1037.0,1360.0,2485.0,469.0,113.0,1272.0,7839.0,0,NaN
2,2,Aaron McKie,957.0,330.0,22594.0,2554.0,44.9,-3.7,14.0,5923.0,...,580.0,2459.0,3039.0,2462.0,1090.0,199.0,1225.0,6636.0,0,NaN
3,3,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,17203.0,...,2589.0,3803.0,6392.0,3254.0,1105.0,187.0,2894.0,26274.0,1,6
4,4,Adrian Smith,796.0,0.0,19411.0,3240.0,32.6,0.0,0.0,7610.0,...,0.0,0.0,1662.0,1829.0,0.0,0.0,0.0,8940.0,0,0
5,5,Al Attles,711.0,0.0,17826.0,2499.0,36.6,0.0,0.0,5545.0,...,0.0,0.0,2463.0,2483.0,0.0,0.0,0.0,6328.0,0,5
6,6,Al Harrington,1132.0,579.0,33118.0,6055.0,50.8,-37.5,6.9,13613.0,...,1745.0,4688.0,6433.0,1900.0,1011.0,292.0,2168.0,15991.0,0,NaN
7,7,Al Horford,646.0,642.0,21535.0,3923.0,70.6,29.5,27.2,7427.0,...,1472.0,4136.0,5608.0,1926.0,534.0,784.0,1031.0,9240.0,0,0
8,8,Al Jefferson,879.0,650.0,25815.0,6050.0,69.4,-2.2,14.7,12136.0,...,2024.0,5523.0,7547.0,1317.0,612.0,1070.0,1190.0,14091.0,0,8
9,9,Albert King,534.0,326.0,12836.0,2709.0,16.4,-19.6,1.8,5875.0,...,760.0,1502.0,2262.0,1171.0,452.0,196.0,1151.0,6470.0,0,2


In [137]:
new_data = new_data.drop(['index'],axis=1)
new_data = new_data.reset_index()

In [138]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

X_train, X_test, Y_train, Y_test = train_test_split(new_data[['TRB', 'AST', 'STL', 'BLK', 'PTS', 'VORP','WS','BPM']], new_data['HOF'], test_size=0.25)


In [147]:
merged = new_data.merge(player_list, on= 'Player')
merged.head()

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,HOF,all_star_appear,URL,From,To,Pos,Ht,Wt,Birth Date,Colleges
0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,...,0,1,greenac01,1986,2001,F-C,9-Jun,220.0,October 4 1963,Oregon State University
1,1,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,6893.0,...,0,NaN,brookaa01,2008,2018,G,Jun-00,161.0,January 14 1985,University of Oregon
2,2,Aaron McKie,957.0,330.0,22594.0,2554.0,44.9,-3.7,14.0,5923.0,...,0,NaN,mckieaa01,1995,2007,G,5-Jun,209.0,October 2 1972,Temple University
3,3,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,17203.0,...,1,6,dantlad01,1977,1991,F-G,5-Jun,208.0,February 28 1956,University of Notre Dame
4,4,Adrian Smith,796.0,0.0,19411.0,3240.0,32.6,0.0,0.0,7610.0,...,0,0,smithad01,1962,1972,G,1-Jun,180.0,October 5 1936,University of Kentucky


In [148]:
merged = merged.drop(['From','To','Ht', 'Wt', 'Birth Date', 'Colleges', 'Pos'],axis=1)

In [149]:
merged

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PTS,HOF,all_star_appear,URL
0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,...,6553.0,10129.0,1469.0,1103.0,562.0,1508.0,12928.0,0,1,greenac01
1,1,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,6893.0,...,1037.0,1360.0,2485.0,469.0,113.0,1272.0,7839.0,0,NaN,brookaa01
2,2,Aaron McKie,957.0,330.0,22594.0,2554.0,44.9,-3.7,14.0,5923.0,...,2459.0,3039.0,2462.0,1090.0,199.0,1225.0,6636.0,0,NaN,mckieaa01
3,3,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,17203.0,...,3803.0,6392.0,3254.0,1105.0,187.0,2894.0,26274.0,1,6,dantlad01
4,4,Adrian Smith,796.0,0.0,19411.0,3240.0,32.6,0.0,0.0,7610.0,...,0.0,1662.0,1829.0,0.0,0.0,0.0,8940.0,0,0,smithad01
5,6,Al Harrington,1132.0,579.0,33118.0,6055.0,50.8,-37.5,6.9,13613.0,...,4688.0,6433.0,1900.0,1011.0,292.0,2168.0,15991.0,0,NaN,harrial01
6,7,Al Horford,646.0,642.0,21535.0,3923.0,70.6,29.5,27.2,7427.0,...,4136.0,5608.0,1926.0,534.0,784.0,1031.0,9240.0,0,0,horfoal01
7,8,Al Jefferson,879.0,650.0,25815.0,6050.0,69.4,-2.2,14.7,12136.0,...,5523.0,7547.0,1317.0,612.0,1070.0,1190.0,14091.0,0,8,jeffeal01
8,9,Albert King,534.0,326.0,12836.0,2709.0,16.4,-19.6,1.8,5875.0,...,1502.0,2262.0,1171.0,452.0,196.0,1151.0,6470.0,0,2,kingal01
9,10,Alex English*,1271.0,753.0,40464.0,11212.0,105.9,20.0,37.4,22149.0,...,4096.0,7143.0,4575.0,1140.0,895.0,3035.0,26931.0,1,11,englial01


In [142]:
merged[merged.HOF==1]

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PTS,HOF,all_star_appear,URL
3,3,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,17203.0,...,3803.0,6392.0,3254.0,1105.0,187.0,2894.0,26274.0,1,6,dantlad01
9,10,Alex English*,1271.0,753.0,40464.0,11212.0,105.9,20.0,37.4,22149.0,...,4096.0,7143.0,4575.0,1140.0,895.0,3035.0,26931.0,1,11,englial01
11,12,Allen Iverson*,1064.0,1042.0,43303.0,9532.0,108.7,20.7,46.0,22369.0,...,3025.0,3839.0,6488.0,2213.0,185.0,3740.0,27457.0,1,7,iversal01
13,14,Alonzo Mourning*,875.0,703.0,26677.0,5216.0,90.9,4.1,23.9,9913.0,...,5098.0,7335.0,964.0,422.0,2430.0,2266.0,14593.0,1,1,mournal01
34,36,Arnie Risen*,577.0,0.0,12690.0,2265.0,45.3,0.0,0.0,6034.0,...,0.0,5011.0,958.0,0.0,0.0,0.0,6638.0,1,NaN,risenar01
36,38,Artis Gilmore*,980.0,503.0,30578.0,5831.0,108.4,23.2,39.1,9751.0,...,6664.0,9372.0,1798.0,485.0,1777.0,2414.0,15844.0,1,6,gilmoar01
40,42,Bailey Howell*,950.0,0.0,30627.0,6515.0,114.7,0.0,0.0,13585.0,...,0.0,9383.0,1853.0,0.0,0.0,0.0,17770.0,1,0,howelba01
46,48,Bernard King*,874.0,547.0,29417.0,7830.0,75.2,12.8,27.5,15109.0,...,3356.0,5060.0,2863.0,866.0,230.0,2791.0,19655.0,1,0,kingbe01
47,49,Bill Bradley*,742.0,0.0,22799.0,3927.0,38.8,-10.1,-0.8,8763.0,...,643.0,2354.0,2533.0,209.0,65.0,0.0,9217.0,1,12,bradlbi01
52,54,Bill Russell*,963.0,0.0,40726.0,5687.0,163.4,0.0,0.0,12930.0,...,0.0,21620.0,4100.0,0.0,0.0,0.0,14522.0,1,0,russebi01


In [143]:
vorp = []
for i in range(len(merged)):
    if merged['VORP'][i] == 0:
        vorp.append(merged['Player'][i])

for i in vorp:
    merged = merged[new_data.Player.str.match(i) == False]

C:\Users\dsran\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [151]:
merged

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PTS,HOF,all_star_appear,URL
0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,...,6553.0,10129.0,1469.0,1103.0,562.0,1508.0,12928.0,0,NaN,greenac01
1,1,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,6893.0,...,1037.0,1360.0,2485.0,469.0,113.0,1272.0,7839.0,0,NaN,brookaa01
2,2,Aaron McKie,957.0,330.0,22594.0,2554.0,44.9,-3.7,14.0,5923.0,...,2459.0,3039.0,2462.0,1090.0,199.0,1225.0,6636.0,0,NaN,mckieaa01
3,3,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,17203.0,...,3803.0,6392.0,3254.0,1105.0,187.0,2894.0,26274.0,1,NaN,dantlad01
4,4,Adrian Smith,796.0,0.0,19411.0,3240.0,32.6,0.0,0.0,7610.0,...,0.0,1662.0,1829.0,0.0,0.0,0.0,8940.0,0,NaN,smithad01
5,6,Al Harrington,1132.0,579.0,33118.0,6055.0,50.8,-37.5,6.9,13613.0,...,4688.0,6433.0,1900.0,1011.0,292.0,2168.0,15991.0,0,NaN,harrial01
6,7,Al Horford,646.0,642.0,21535.0,3923.0,70.6,29.5,27.2,7427.0,...,4136.0,5608.0,1926.0,534.0,784.0,1031.0,9240.0,0,NaN,horfoal01
7,8,Al Jefferson,879.0,650.0,25815.0,6050.0,69.4,-2.2,14.7,12136.0,...,5523.0,7547.0,1317.0,612.0,1070.0,1190.0,14091.0,0,NaN,jeffeal01
8,9,Albert King,534.0,326.0,12836.0,2709.0,16.4,-19.6,1.8,5875.0,...,1502.0,2262.0,1171.0,452.0,196.0,1151.0,6470.0,0,NaN,kingal01
9,10,Alex English*,1271.0,753.0,40464.0,11212.0,105.9,20.0,37.4,22149.0,...,4096.0,7143.0,4575.0,1140.0,895.0,3035.0,26931.0,1,NaN,englial01


In [196]:
# merged['all_star_appear'] = np.nan
# merged['nba_champ'] = np.nan
# merged['all_defensive']= np.nan
# merged['all_star_mvp'] = np.nan
# merged['league_mvp'] = np.nan
# merged['scoring_champ'] = np.nan
# merged['all_nba'] = np.nan
# merged['finals_mvp'] = np.nan
# merged['def_poy'] = np.nan
# merged['stl_champ'] = np.nan
# merged['ast_champ'] = np.nan
# merged['trb_champ'] = np.nan
# merged['blk_champ'] = np.nan

In [194]:
import re
import requests
for counter, i in enumerate(merged['Player']):
    if counter == 3913:
        url = 'https://www.basketball-reference.com/players/h/hilarne01.html'
    else:
        stem = merged[merged['Player'].str.contains(i)].reset_index().at[0,'URL']
        url = 'https://www.basketball-reference.com/players/'+stem[0]+'/'+stem+'.html'
    page = requests.get(url)
    soup = bs(page.content,'html.parser')
    if soup.find("ul",id='bling') is None:
        new_data.iloc[counter,25:] = 0
        continue
    else:
        body = soup.find("ul", id='bling').findAll('a')
    for award in body:
        result = re.sub('[^0-9]','', award.getText())
        if('BLK Champ' in award.getText()):
            merged.at[merged.index[counter], 'blk_champ'] = result
            continue

In [206]:
merged

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,all_star_mvp,league_mvp,scoring_champ,all_nba,finals_mvp,def_poy,stl_champ,ast_champ,trb_champ,blk_champ
0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,6893.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Aaron McKie,957.0,330.0,22594.0,2554.0,44.9,-3.7,14.0,5923.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,17203.0,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Adrian Smith,796.0,0.0,19411.0,3240.0,32.6,0.0,0.0,7610.0,...,196566.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Al Harrington,1132.0,579.0,33118.0,6055.0,50.8,-37.5,6.9,13613.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Al Horford,646.0,642.0,21535.0,3923.0,70.6,29.5,27.2,7427.0,...,NaN,NaN,NaN,201011.0,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Al Jefferson,879.0,650.0,25815.0,6050.0,69.4,-2.2,14.7,12136.0,...,NaN,NaN,NaN,201314.0,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Albert King,534.0,326.0,12836.0,2709.0,16.4,-19.6,1.8,5875.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Alex English*,1271.0,753.0,40464.0,11212.0,105.9,20.0,37.4,22149.0,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged = merged.drop(['From','To','Ht', 'Wt', 'Birth Date', 'Colleges', 'Pos'],axis=1)

In [200]:
# merged.to_csv('export.csv')

In [238]:
kal = merged.loc[merged.Player=='Kareem Abdul-Jabbar*']
kal

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,all_star_mvp,league_mvp,scoring_champ,all_nba,finals_mvp,def_poy,stl_champ,ast_champ,trb_champ,blk_champ
368,373,Kareem Abdul-Jabbar*,1560.0,625.0,57446.0,15837.0,273.3,85.3,85.9,28307.0,...,NaN,6.0,NaN,15.0,2.0,NaN,NaN,NaN,197576.0,4.0


In [318]:
mvp = merged.loc[merged.Player=="Chris Paul"]
mvp

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,all_star_mvp,league_mvp,scoring_champ,all_nba,finals_mvp,def_poy,stl_champ,ast_champ,trb_champ,blk_champ
123,125,Chris Paul,834.0,834.0,29646.0,5493.0,154.7,89.2,71.5,11614.0,...,201213.0,NaN,NaN,8.0,NaN,NaN,6.0,4.0,NaN,NaN


In [314]:
# Add 'league_mvp' manually
# merged.loc[merged.Player=='Bob Pettit*' , 'league_mvp'] = 2
# merged.loc[merged.Player=='Bob Cousy*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='Bill Russell*' , 'league_mvp'] = 5
# merged.loc[merged.Player=='Wilt Chamberlain*' , 'league_mvp'] = 4
# merged.loc[merged.Player=='Oscar Robertson*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='Wes Unseld*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='Willis Reed*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='Kareem Abdul-Jabbar*' , 'league_mvp'] = 6
# merged.loc[merged.Player=='Bob McAdoo*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='Bill Walton*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='Bob Pettit*' , 'league_mvp'] = 2
# merged.loc[merged.Player=='Moses Malone*' , 'league_mvp'] = 3
# merged.loc[merged.Player=='Julius Erving*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='Larry Bird*' , 'league_mvp'] = 3
# merged.loc[merged.Player=='Magic Johnson*' , 'league_mvp'] = 3
# merged.loc[merged.Player=='Michael Jordan*' , 'league_mvp'] = 5
# merged.loc[merged.Player=='Charles Barkley*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='Hakeem Olajuwon*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='David Robinson*' , 'league_mvp'] = 1
# merged.loc[merged.Player=='Karl Malone*' , 'league_mvp'] = 2
# merged.loc[merged.Player=="Shaquille O'Neal*" , 'league_mvp'] = 1
# merged.loc[merged.Player=="Allen Iverson*" , 'league_mvp'] = 1
# merged.loc[merged.Player=="Tim Duncan" , 'league_mvp'] = 2
# merged.loc[merged.Player=="Kevin Garnett" , 'league_mvp'] = 1
# merged.loc[merged.Player=="Steve Nash" , 'league_mvp'] = 2
# merged.loc[merged.Player=="Dirk Nowitzki" , 'league_mvp'] = 1
# merged.loc[merged.Player=="Kobe Bryant" , 'league_mvp'] = 1
# merged.loc[merged.Player=="LeBron James" , 'league_mvp'] = 4
# merged.loc[merged.Player=="Derrick Rose" , 'league_mvp'] = 1
# merged.loc[merged.Player=="Kevin Durant" , 'league_mvp'] = 1
# merged.loc[merged.Player=="Stephen Curry" , 'league_mvp'] = 2
# merged.loc[merged.Player=="Russell Westbrook" , 'league_mvp'] = 1
# merged.loc[merged.Player=="James Harden" , 'league_mvp'] = 1
# merged.loc[merged.Player=="Giannis Antetokounmpo" , 'league_mvp'] = 1

In [317]:
merged.to_csv('export2.csv')

In [321]:
# Now my dataset is complete with all the statistics
# Create new dataframes for Hall of Famers and non-HOF
hall_of_fame = merged[merged.HOF == 1]
hall_of_fame

not_hof = merged[merged.HOF == 0]
not_hof

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,all_star_mvp,league_mvp,scoring_champ,all_nba,finals_mvp,def_poy,stl_champ,ast_champ,trb_champ,blk_champ
0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,6893.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Aaron McKie,957.0,330.0,22594.0,2554.0,44.9,-3.7,14.0,5923.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Adrian Smith,796.0,0.0,19411.0,3240.0,32.6,0.0,0.0,7610.0,...,196566.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Al Harrington,1132.0,579.0,33118.0,6055.0,50.8,-37.5,6.9,13613.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Al Horford,646.0,642.0,21535.0,3923.0,70.6,29.5,27.2,7427.0,...,NaN,NaN,NaN,201011.0,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Al Jefferson,879.0,650.0,25815.0,6050.0,69.4,-2.2,14.7,12136.0,...,NaN,NaN,NaN,201314.0,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Albert King,534.0,326.0,12836.0,2709.0,16.4,-19.6,1.8,5875.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,11,Allan Houston,839.0,727.0,28311.0,5337.0,55.2,-13.4,9.9,12017.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,13,Allen Leavell,700.0,244.0,16248.0,2549.0,29.8,-19.2,3.5,5666.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [322]:
# Reset index for both new dataframes
hall_of_fame = hall_of_fame.reset_index()
not_hof = not_hof.reset_index()

In [330]:
hall_of_fame

,level_0,index,Player,G,GS,MP,FG,WS,BPM,VORP,...,all_star_mvp,league_mvp,scoring_champ,all_nba,finals_mvp,def_poy,stl_champ,ast_champ,trb_champ,blk_champ
0,3,3,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,10,Alex English*,1271.0,753.0,40464.0,11212.0,105.9,20.0,37.4,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11,12,Allen Iverson*,1064.0,1042.0,43303.0,9532.0,108.7,20.7,46.0,...,2.0,1.0,0.0,7.0,0.0,0.0,3.0,0.0,0.0,0.0
3,13,14,Alonzo Mourning*,875.0,703.0,26677.0,5216.0,90.9,4.1,23.9,...,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0
4,34,36,Arnie Risen*,577.0,0.0,12690.0,2265.0,45.3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,36,38,Artis Gilmore*,980.0,503.0,30578.0,5831.0,108.4,23.2,39.1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,40,42,Bailey Howell*,950.0,0.0,30627.0,6515.0,114.7,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,46,48,Bernard King*,874.0,547.0,29417.0,7830.0,75.2,12.8,27.5,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
8,47,49,Bill Bradley*,742.0,0.0,22799.0,3927.0,38.8,-10.1,-0.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,52,54,Bill Russell*,963.0,0.0,40726.0,5687.0,163.4,0.0,0.0,...,1.0,5.0,0.0,11.0,0.0,0.0,0.0,0.0,4.0,0.0


In [332]:
not_hof

,level_0,index,Player,G,GS,MP,FG,WS,BPM,VORP,...,all_star_mvp,league_mvp,scoring_champ,all_nba,finals_mvp,def_poy,stl_champ,ast_champ,trb_champ,blk_champ
0,0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,Aaron McKie,957.0,330.0,22594.0,2554.0,44.9,-3.7,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,4,Adrian Smith,796.0,0.0,19411.0,3240.0,32.6,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,6,Al Harrington,1132.0,579.0,33118.0,6055.0,50.8,-37.5,6.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,7,Al Horford,646.0,642.0,21535.0,3923.0,70.6,29.5,27.2,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,8,Al Jefferson,879.0,650.0,25815.0,6050.0,69.4,-2.2,14.7,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,9,Albert King,534.0,326.0,12836.0,2709.0,16.4,-19.6,1.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,10,11,Allan Houston,839.0,727.0,28311.0,5337.0,55.2,-13.4,9.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,12,13,Allen Leavell,700.0,244.0,16248.0,2549.0,29.8,-19.2,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [335]:
# Fill in 'NaN' with '0'
hall_of_fame = hall_of_fame.fillna(0)
not_hof = not_hof.fillna(0)
merged = merged.fillna(0)

In [329]:
# Looking at the datasets, if a player only won an award once
# The year they won the award is returned
# Write a 'for' loop to change the year to '1' for all columns after
# Looking again at the data it would be the 28th column on
for i in hall_of_fame.columns[28:]:
    hall_of_fame[i][hall_of_fame[i] > 100000] = 1

C:\Users\dsran\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [331]:
for i in not_hof.columns[28:]:
    not_hof[i][not_hof[i] > 100000] = 1

C:\Users\dsran\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [341]:
for i in merged.columns[27:]:
    merged[i][merged[i] > 100000] = 1

C:\Users\dsran\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [344]:
merged = merged.fillna(0)

In [340]:
# Export csvs
hall_of_fame.to_csv('hall_of_fame.csv')
not_hof.to_csv('not_hof.csv')
merged.to_csv('all_players_data.csv')

In [346]:
merged.to_csv('merged.csv')

In [347]:
merged

,index,Player,G,GS,MP,FG,WS,BPM,VORP,FGA,...,all_star_mvp,league_mvp,scoring_champ,all_nba,finals_mvp,def_poy,stl_champ,ast_champ,trb_champ,blk_champ
0,0,A.C. Green,1361.0,905.0,39044.0,4778.0,104.2,-1.5,23.0,9686.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Aaron Brooks,797.0,226.0,17088.0,2829.0,23.2,-38.5,0.4,6893.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Aaron McKie,957.0,330.0,22594.0,2554.0,44.9,-3.7,14.0,5923.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Adrian Dantley*,1107.0,613.0,39506.0,9217.0,154.0,43.7,50.0,17203.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Adrian Smith,796.0,0.0,19411.0,3240.0,32.6,0.0,0.0,7610.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Al Harrington,1132.0,579.0,33118.0,6055.0,50.8,-37.5,6.9,13613.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,Al Horford,646.0,642.0,21535.0,3923.0,70.6,29.5,27.2,7427.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,Al Jefferson,879.0,650.0,25815.0,6050.0,69.4,-2.2,14.7,12136.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Albert King,534.0,326.0,12836.0,2709.0,16.4,-19.6,1.8,5875.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,Alex English*,1271.0,753.0,40464.0,11212.0,105.9,20.0,37.4,22149.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
